<a href="https://colab.research.google.com/github/BradenAnderson/sales-predictions/blob/main/04_Feature_Engineering_and_Modeling_Pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Braden Anderson
## Sales Predictions Project
## Feature Engineering and Modeling Part 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import missingno
import pickle
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, BaggingRegressor 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score, mean_squared_error, SCORERS
from sklearn.pipeline import Pipeline


%matplotlib inline

In [ ]:
# Importing the original data (NOT what was previously cleaned).
filename = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/sales_predictions.csv'

sales_df = pd.read_csv(filename)

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Consider any products with item_visibility = 0 as having missing information. (Not realistic that the visibility of any item in a store is actually zero).
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Calculate the average visibility for each unique value of 'Item_Identifier'
average_visibility = sales_df.groupby(by='Item_Identifier')['Item_Visibility'].mean()

# Create a filter to find all the rows where the visibility value is zero.
missing_visibility_filter = (sales_df['Item_Visibility'] == 0)

# Change all values of visibility=0 to the average visibility value for that item type. 
sales_df.loc[missing_visibility_filter, 'Item_Visibility'] = sales_df.loc[missing_visibility_filter, 'Item_Identifier'].apply(lambda x: average_visibility[x])

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Notice that the Item_Identifier column contains many categories, some of which do not have very many examples (instances) in the dataset. 
# 
# Combine the Item_Identifier column into broader categories, using the fact that the first two letters in each item identifier
# point to which category the item falls into (FD = Food, DR = Drink, NC = Non-Consumeable).
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Create a new column "Item_Type_Binned" that contains the first two characters in the Item_Identifier string.
sales_df['Item_Id_Binned'] = sales_df['Item_Identifier'].apply(lambda x : x[0:2])


# Expand the two letter code in the "Item_Type_Binned" column (created above) into a more obvious category name.
sales_df['Item_Id_Binned'] = sales_df['Item_Id_Binned'].map({'FD' : 'Food', 'NC' : 'Non-Consumable', 'DR' : 'Drinks'})

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Consider that "Years of operation" may have more of a relationship with sales than "Establishment_year".
#
# For perspective, both of these features would be ordinal (an ordering exists both amongst years established and years of operation), however an ordinal encoder would
# default to giving the oldest store the smallest numeric value (1985 = 0) if the "Establishment_year" feature is used. If the "years of operation" feature is used,
# now the oldest store has the largest value, and as such an ordinal encoder would then give the oldest store the largest numeric value. This difference may be subtle
# but could affect the way a ML algorithm fits a model to these features. 
#
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Create a new column 'Years of operation' that equals the number of years prior to 2013 that this store was established.
sales_df['Years_of_operation'] = 2013 - sales_df['Outlet_Establishment_Year']

In [ ]:
# Make category names in the Item_Fat_Content feature consistent.
sales_df['Item_Fat_Content'] = sales_df['Item_Fat_Content'].replace({'LF' : 'Low Fat', 'reg' : 'Regular', 'low fat' : 'Low Fat'})

In [ ]:
# Some "Non-Consumable" item types have a fat content listed. For all "Non-Consumable" items, change the fat content value to 
# a new category called "Not_Edible".
sales_df.loc[sales_df['Item_Id_Binned'] == 'Non-Consumable', 'Item_Fat_Content'] = 'Not_Edible'

#Imputing

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# Imputing the missing weight values using the average item weight for that item type. 
# -------------------------------------------------------------------------------------------------------------------------

# Create groups of each unique item identifier. For each group, calculate the average item weight.
average_weight_per_item = sales_df.groupby(by="Item_Identifier")['Item_Weight'].mean()

# Create a filter to easily grab all the rows where the Item_Weight value is missing.
missing_weight_filter = (sales_df['Item_Weight'].isna())

# Set each missing Item_Weight equal to the average item weight for that Item_Identifier.
# For example, for item 'DRA24' the weight value with be mapped to average_weight_per_item['DRA24'], which is 19.35 per the averages we calculated two lines above.
sales_df.loc[missing_weight_filter, 'Item_Weight'] = sales_df.loc[missing_weight_filter, 'Item_Identifier'].apply(lambda Item_ID : average_weight_per_item[Item_ID] )

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# Imputing the missing Outlet_Size values with the mode (most frequent) outlet size value for that outlet type.
# -------------------------------------------------------------------------------------------------------------------------

# Create groups for each unique value of Outlet_Type. For each outlet_type group, calculate the most frequent outlet_size. 
most_frequent_size_per_type = sales_df.groupby(by="Outlet_Type")['Outlet_Size'].agg(pd.Series.mode)

# Create a filter for easily accessing all the rows where the Outlet_Size value is missing.
missing_outlet_size = sales_df['Outlet_Size'].isna()

# impute the missing values in the Outlet_Size column with the most frequent Outlet_Size for that Outlet_Type
sales_df.loc[missing_outlet_size, 'Outlet_Size'] = sales_df.loc[missing_outlet_size, 'Outlet_Type'].apply(lambda store_type : most_frequent_size_per_type[store_type])

In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------
# There are still four missing values in the Item_Weight column. This is because we imputed the missing Item_Weight values using the average weight per item type, and 
# it turns out for these four items there is only a single instance of that item type, therefore no other information existed to compute an average from! 
# ---------------------------------------------------------------------------------------------------------------------------------------------------------------------

sales_df.isna().sum()

Item_Identifier              0
Item_Weight                  4
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
Item_Id_Binned               0
Years_of_operation           0
dtype: int64

In [ ]:
# Impute the four remaining missing Item_Weight values using the average Item_Weight for that Item_Type. 
average_weights_per_type = sales_df.groupby('Item_Type')['Item_Weight'].mean()

# Filter to easily select the four rows that still have missing Item_Weight values.
missing_weights = sales_df['Item_Weight'].isna()

# Imputing the four remaining missing Item_Weight values with the average weight value for that item type. 
sales_df.loc[missing_weights, 'Item_Weight'] = sales_df.loc[missing_weights, 'Item_Type'].apply(lambda item_type : average_weights_per_type[item_type])

In [ ]:
sales_df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
Item_Id_Binned               0
Years_of_operation           0
dtype: int64

In [ ]:
filepath = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/Sales_Training_Data_Cleaned_Imputed.csv'

sales_df.to_csv(path_or_buf=filepath, index=False)

sales_df.drop(columns=['Item_Identifier', 'Outlet_Establishment_Year'], inplace=True)

In [ ]:
'''
nominal_categorical_feat = ['Item_Type', 'Outlet_Identifier', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Id_Binned']
ordinal_categorical_feat = ['Item_Fat_Content', 'Outlet_Size', 'Years_of_operation']
numeric_feat = ['Item_Weight', 'Item_Visibility', 'Item_MRP']

X = sales_df.loc[:, sales_df.columns != 'Item_Outlet_Sales']
y = sales_df.loc[:, 'Item_Outlet_Sales']

preprocess = ColumnTransformer(transformers=[("std_scaler", StandardScaler(), numeric_feat),
                                             ("one_hot_encoder", OneHotEncoder(drop="first", sparse=False), nominal_categorical_feat),
                                             ("ordinal_encoder", OrdinalEncoder(), ordinal_categorical_feat)],
                               remainder='passthrough')


num_trees = [1, 5, 10, 50, 100, 150, 200]
max_depth = [None, 7]

column_list = list(range(5,38))
num_neighbors = list(range(1, 26))

random_forest = RandomForestRegressor(random_state=42, oob_score=True)
bagged_trees = BaggingRegressor(random_state=42, oob_score=True)

reg_strategy = TransformedTargetRegressor()
bcox_transformer = PowerTransformer(method='box-cox')

model_pipeline = Pipeline([("preprocessing", preprocess),
                           ('feature_selector', SelectKBest(f_regression)),
                           ('regress', reg_strategy)])

# -----------------------------------------------------------------------------------------------------------------------------------------------
# The dictionary key with double unscore (feature_selector__K) is a special syntax that allows us tell GridSearch to vary the "k" parameter
# of the object named "feature_selector" (which in our case is a SelectKBest object) when search for the best set of parameters.
# -----------------------------------------------------------------------------------------------------------------------------------------------
parameter_grid = [{'feature_selector__k': column_list,
                   'regress__regressor' : [bagged_trees],           
                   'regress__regressor__n_estimators' : num_trees,
                   'regress__regressor__base_estimator' : [DecisionTreeRegressor()],
                   'regress__regressor__base_estimator__max_depth' : max_depth,  
                   'regress__transformer' : [None, bcox_transformer]},
                  {'feature_selector__k': column_list,                       
                   'regress__regressor' : [random_forest],           
                   'regress__regressor__n_estimators' : num_trees,
                   'regress__regressor__max_depth' : max_depth,  
                   'regress__transformer' : [None, bcox_transformer]},
                  {'feature_selector__k' : column_list,                         # Linear Regression with all possible numbers of "best features" evaluted.
                   'regress__regressor' : [LinearRegression(), Lasso()],         # Target is box-cox transformed to zero mean and unit variance
                   'regress__transformer' : [None, bcox_transformer]},
                  {'feature_selector__k' : column_list,                     # Same as above, only difference is the target is target is scaled using box-cox
                   'regress__regressor' : [KNeighborsRegressor()],
                   'regress__regressor__n_neighbors' : num_neighbors,
                   'regress__regressor__weights' : ['uniform', 'distance'],
                   'regress__transformer' : [None, bcox_transformer]}]



score_types = {'r2' :'r2', 'MSE' : 'neg_mean_squared_error'}

gs = GridSearchCV(estimator=model_pipeline, param_grid=parameter_grid, scoring=score_types, refit='MSE', cv=5, n_jobs=-1)

gs.fit(X, y)

PATH = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/'
gridsearch_result_filename = 'gridsearch_models_2.pkl'
full_path = PATH + gridsearch_result_filename
with open(full_path, 'wb') as file:
  pickle.dump(gs, file)
'''

'\nnominal_categorical_feat = [\'Item_Type\', \'Outlet_Identifier\', \'Outlet_Location_Type\', \'Outlet_Type\', \'Item_Id_Binned\']\nordinal_categorical_feat = [\'Item_Fat_Content\', \'Outlet_Size\', \'Years_of_operation\']\nnumeric_feat = [\'Item_Weight\', \'Item_Visibility\', \'Item_MRP\']\n\nX = sales_df.loc[:, sales_df.columns != \'Item_Outlet_Sales\']\ny = sales_df.loc[:, \'Item_Outlet_Sales\']\n\npreprocess = ColumnTransformer(transformers=[("std_scaler", StandardScaler(), numeric_feat),\n                                             ("one_hot_encoder", OneHotEncoder(drop="first", sparse=False), nominal_categorical_feat),\n                                             ("ordinal_encoder", OrdinalEncoder(), ordinal_categorical_feat)],\n                               remainder=\'passthrough\')\n\n\nnum_trees = [1, 5, 10, 50, 100, 150, 200]\nmax_depth = [None, 7]\n\ncolumn_list = list(range(5,38))\nnum_neighbors = list(range(1, 26))\n\nrandom_forest = RandomForestRegressor(random_state

In [ ]:
full_path = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/gridsearch_models_2.pkl'

with open(full_path, 'rb') as file:
  gs_results = pickle.load(file)

search_results = gs_results.cv_results_
top_estimator = gs_results.best_estimator_
top_score = gs_results.best_score_
top_parameters = gs_results.best_params_

In [ ]:
top_parameters

{'feature_selector__k': 34,
 'regress__regressor': RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                       max_depth=7, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=True,
                       random_state=42, verbose=0, warm_start=False),
 'regress__regressor__max_depth': 7,
 'regress__regressor__n_estimators': 100,
 'regress__transformer': None}

In [ ]:
gs_result_df = pd.DataFrame(search_results)

list(gs_result_df.columns)

['mean_fit_time',
 'std_fit_time',
 'mean_score_time',
 'std_score_time',
 'param_feature_selector__k',
 'param_regress__regressor',
 'param_regress__regressor__base_estimator',
 'param_regress__regressor__base_estimator__max_depth',
 'param_regress__regressor__n_estimators',
 'param_regress__transformer',
 'param_regress__regressor__max_depth',
 'param_regress__regressor__n_neighbors',
 'param_regress__regressor__weights',
 'params',
 'split0_test_r2',
 'split1_test_r2',
 'split2_test_r2',
 'split3_test_r2',
 'split4_test_r2',
 'mean_test_r2',
 'std_test_r2',
 'rank_test_r2',
 'split0_test_MSE',
 'split1_test_MSE',
 'split2_test_MSE',
 'split3_test_MSE',
 'split4_test_MSE',
 'mean_test_MSE',
 'std_test_MSE',
 'rank_test_MSE']

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------------
# Perform Data Cleaning on the GridSearchCV output
#------------------------------------------------------------------------------------------------------------------------------------------------

gs_result_df = pd.DataFrame(search_results)

results_simplified_df= gs_result_df.loc[:,:].sort_values(by=['rank_test_MSE'], ignore_index=True)

results_simplified_df.rename(columns= {'param_feature_selector__k' : 'num_features_in_model',
                                       'param_regress__regressor' : 'model_type',
                                       'param_regress__transformer' : 'target_transformation',
                                       'param_regress__regressor__n_neighbors' : "Num_Nearest_Neighbors",
                                       'param_regress__regressor__weights' : "Neighbors_Score_Strategy",
                                       'param_regress__regressor__max_depth' : 'rand_forest_max_depth',
                                       'param_regress__regressor__n_estimators' : 'Number_of_Trees',
                                       'param_regress__regressor__base_estimator__max_depth' : 'bagged_tree_max_depth'}, inplace=True)

results_simplified_df = results_simplified_df.loc[:, ['num_features_in_model', 'model_type', 'target_transformation', 'Num_Nearest_Neighbors',
                                                      'Neighbors_Score_Strategy', 'rand_forest_max_depth', 'Number_of_Trees', 'bagged_tree_max_depth',
                                                      'mean_test_r2', 'rank_test_r2', 'mean_test_MSE', 'rank_test_MSE']]

# For easier viewing, combine 'rand_forest_max_depth' and 'bagged_tree_max_depth" into a single column called "tree_depth".
results_simplified_df['tree_depth'] = results_simplified_df['rand_forest_max_depth'].fillna(value=results_simplified_df['bagged_tree_max_depth'])
results_simplified_df['tree_depth'] = results_simplified_df['tree_depth'].fillna(value='No_Limit')
results_simplified_df.drop(columns=['rand_forest_max_depth', 'bagged_tree_max_depth'], inplace=True)

# Indicate if the model recieved a 'Box-cox' transformation of the target variable, or if not then list 'No_transformation'.
results_simplified_df['target_transformation'] = results_simplified_df['target_transformation'].fillna(value="No_Transformation")
results_simplified_df.loc[ (results_simplified_df['target_transformation'] != 'No_Transformation') , 'target_transformation'] = "Box-Cox"

# For easier viewing, trim model_type down so it only shows the name of the model used.
results_simplified_df['model_type'] = results_simplified_df['model_type'].astype(str)
results_simplified_df['model_type'] = results_simplified_df['model_type'].map(lambda model_string : model_string.split('(')[0])

results_simplified_df['Num_Nearest_Neighbors'] = results_simplified_df['Num_Nearest_Neighbors'].fillna(value='Not_Applicable')
results_simplified_df['Neighbors_Score_Strategy'] = results_simplified_df['Neighbors_Score_Strategy'].fillna(value='Not_Applicable')
results_simplified_df['Number_of_Trees'] = results_simplified_df['Number_of_Trees'].fillna(value='Not_Applicable')

# Negative MSE is not very interpretable. Take the absolute value and square root to get a more meaningful number. 
results_simplified_df['RMSE'] = results_simplified_df['mean_test_MSE'].abs().pow(1./2)

pd.set_option('display.max_rows', 500)

results_simplified_df.head(500)


,num_features_in_model,model_type,target_transformation,Num_Nearest_Neighbors,Neighbors_Score_Strategy,Number_of_Trees,mean_test_r2,rank_test_r2,mean_test_MSE,rank_test_MSE,tree_depth,RMSE
0,34,RandomForestRegressor,No_Transformation,Not_Applicable,Not_Applicable,100,0.594729,1,-1.177959e+06,1,7,1085.338347
1,34,RandomForestRegressor,No_Transformation,Not_Applicable,Not_Applicable,200,0.594614,2,-1.178265e+06,2,7,1085.479227
2,34,RandomForestRegressor,No_Transformation,Not_Applicable,Not_Applicable,150,0.594592,5,-1.178333e+06,3,7,1085.510267
3,37,RandomForestRegressor,No_Transformation,Not_Applicable,Not_Applicable,100,0.594595,4,-1.178374e+06,4,7,1085.529219
4,31,RandomForestRegressor,No_Transformation,Not_Applicable,Not_Applicable,100,0.594600,3,-1.178378e+06,5,7,1085.531324
5,33,RandomForestRegressor,No_Transformation,Not_Applicable,Not_Applicable,100,0.594570,6,-1.178463e+06,6,7,1085.570219
6,33,RandomForestRegressor,No_Transformation,Not_Applicable,Not_Applicable,200,0.594533,7,-1.178548e+06,7,7,1085.609329
7,33,BaggingRegressor,No_Transformation,Not_Applicable,Not_Applicable,200,0.594519,9,-1.178582e+06,8,7,1085.625256
8,33,RandomForestRegressor,No_Transformation,Not_Applicable,Not_Applicable,150,0.594522,8,-1.178600e+06,9,7,1085.633460
9,37,RandomForestRegressor,No_Transformation,Not_Applicable,Not_Applicable,150,0.594494,10,-1.178657e+06,10,7,1085.659491


# Preparing the test set for evaluation 

In [ ]:
# Loading the test file.
filename = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Week_6/test_AbJTz2l.csv'

test_df = pd.read_csv(filename)

submission_columns_df = test_df[['Item_Identifier', 'Outlet_Identifier']]


## Perform the feature engineering and imputation methods on the test set
## so it can be evaluated by the fitted machine learning model.

In [ ]:
# Average visibility for each unique value of 'Item_Identifier', calculated during previous training data preparation.
average_visibility

Item_Identifier
DRA12    0.031956
DRA24    0.048062
DRA59    0.134718
DRB01    0.082126
DRB13    0.008002
           ...   
NCZ30    0.024956
NCZ41    0.051623
NCZ42    0.009044
NCZ53    0.027775
NCZ54    0.080426
Name: Item_Visibility, Length: 1559, dtype: float64

In [ ]:
# Create a filter to find all the rows where the visibility value is zero.
missing_visibility_filter = (test_df['Item_Visibility'] == 0)

# Change all values of visibility=0 to the average visibility value for that item type. 
test_df.loc[missing_visibility_filter, 'Item_Visibility'] = test_df.loc[missing_visibility_filter, 'Item_Identifier'].apply(lambda x: average_visibility[x])

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Binning Test Data Item_Id
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Create a new column "Item_Type_Binned" that contains the first two characters in the Item_Identifier string.
test_df['Item_Id_Binned'] = test_df['Item_Identifier'].apply(lambda x : x[0:2])


# Expand the two letter code in the "Item_Type_Binned" column (created above) into a more obvious category name.
test_df['Item_Id_Binned'] = test_df['Item_Id_Binned'].map({'FD' : 'Food', 'NC' : 'Non-Consumable', 'DR' : 'Drinks'})

In [ ]:
# Create a new column 'Years of operation' that equals the number of years prior to 2013 that this store was established.
test_df['Years_of_operation'] = 2013 - test_df['Outlet_Establishment_Year']

In [ ]:
# Make category names in the Item_Fat_Content feature consistent.
test_df['Item_Fat_Content'] = test_df['Item_Fat_Content'].replace({'LF' : 'Low Fat', 'reg' : 'Regular', 'low fat' : 'Low Fat'})

# Some "Non-Consumable" item types have a fat content listed. For all "Non-Consumable" items, change the fat content value to 
# a new category called "Not_Edible".
test_df.loc[sales_df['Item_Id_Binned'] == 'Non-Consumable', 'Item_Fat_Content'] = 'Not_Edible'

# Ensure the only values in the 'Item_Fat_Content' column are now "Low Fat", "Regular" and "Not_Edible"
# test_df['Item_Fat_Content'].value_counts()

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# Imputing the missing weight values using the average item weight for that item type. 
# -------------------------------------------------------------------------------------------------------------------------

# Create a filter to easily grab all the rows where the Item_Weight value is missing.
missing_weight_filter = (test_df['Item_Weight'].isna())

# Set each missing Item_Weight equal to the average item weight for that Item_Identifier (as calculated using the training data set).
# For example, for item 'DRA24' the weight value with be mapped to average_weight_per_item['DRA24'], which is 19.35 per the averages we calculated two lines above.
test_df.loc[missing_weight_filter, 'Item_Weight'] = test_df.loc[missing_weight_filter, 'Item_Identifier'].apply(lambda Item_ID : average_weight_per_item[Item_ID] )

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# Imputing the missing Outlet_Size values with the mode (most frequent) outlet size value for that outlet type.
# -------------------------------------------------------------------------------------------------------------------------

# Create a filter for easily accessing all the rows where the Outlet_Size value is missing.
missing_outlet_size = test_df['Outlet_Size'].isna()

# impute the missing values in the Outlet_Size column with the most frequent Outlet_Size for that Outlet_Type
test_df.loc[missing_outlet_size, 'Outlet_Size'] = test_df.loc[missing_outlet_size, 'Outlet_Type'].apply(lambda store_type : most_frequent_size_per_type[store_type])

In [ ]:
test_df.isna().sum()

Item_Identifier              0
Item_Weight                  1
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Id_Binned               0
Years_of_operation           0
dtype: int64

In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------------------------------------
# There is still a single missing weight value. Just as we saw when processing the training set, this occurs when there is a missing weight value
# and there is no other items with that item_ID to average the weights of for imputation. In this case, we will impute the weight with the average
# weight for that item_type (a broader but still relevant category), as we did before.
# ---------------------------------------------------------------------------------------------------------------------------------------------------------

# Filter to easily select the four rows that still have missing Item_Weight values.
missing_weights = test_df['Item_Weight'].isna()

# Imputing the four remaining missing Item_Weight values with the average weight value for that item type. 
test_df.loc[missing_weights, 'Item_Weight'] = test_df.loc[missing_weights, 'Item_Type'].apply(lambda item_type : average_weights_per_type[item_type])

In [ ]:
test_df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Id_Binned               0
Years_of_operation           0
dtype: int64

In [ ]:
filepath = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/Sales_Test_Data_Cleaned_TrainImpute.csv'

test_df.to_csv(path_or_buf=filepath, index=False)

# Dropping the columns that were not used when training the machine learning model.
test_df.drop(columns=['Item_Identifier', 'Outlet_Establishment_Year'], inplace=True)

In [ ]:
# Make sales predictions using the test data.
test_predictions = gs_results.predict(test_df)

In [ ]:
# Convert the predictions to a dataframe.
predictions_df = pd.DataFrame(test_predictions)
predictions_df.rename(columns={0: 'Item_Outlet_Sales'}, inplace=True)

In [ ]:
# Prepare the prediction for output.
full_submission_df = pd.concat([submission_columns_df, predictions_df], axis=1)

In [ ]:
# Output the predictions to a csv file. 
filepath = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/Anderson_Sales_Prediction_Submission.csv'

full_submission_df.to_csv(path_or_buf=filepath, index=False)

# Submission 2 - imputing using all the available information

In [ ]:
# Loading the test file.
filename = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Week_6/test_AbJTz2l.csv'

test_df = pd.read_csv(filename)

submission_columns_df = test_df[['Item_Identifier', 'Outlet_Identifier']]

In [ ]:
# Load the training file (version that has not been cleaned, and missings have not been imputed). 
filename = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/sales_predictions.csv'

sales_df = pd.read_csv(filename)

In [ ]:
# Combine the test and training file. This will allow us to easily use the data from both when imputing values in the test set.
combined_df = pd.concat([sales_df, test_df])

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Consider any products with item_visibility = 0 as having missing information. (Not realistic that the visibility of any item in a store is actually zero).
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Calculate the average visibility for each unique value of 'Item_Identifier'
average_visibility = combined_df.groupby(by='Item_Identifier')['Item_Visibility'].mean()

# Create a filter to find all the rows where the visibility value is zero.
missing_visibility_filter = (test_df['Item_Visibility'] == 0)

# Change all values of visibility=0 to the average visibility value for that item type. 
test_df.loc[missing_visibility_filter, 'Item_Visibility'] = test_df.loc[missing_visibility_filter, 'Item_Identifier'].apply(lambda x: average_visibility[x])

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Binning Test Data Item_Id
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Create a new column "Item_Type_Binned" that contains the first two characters in the Item_Identifier string.
test_df['Item_Id_Binned'] = test_df['Item_Identifier'].apply(lambda x : x[0:2])


# Expand the two letter code in the "Item_Type_Binned" column (created above) into a more obvious category name.
test_df['Item_Id_Binned'] = test_df['Item_Id_Binned'].map({'FD' : 'Food', 'NC' : 'Non-Consumable', 'DR' : 'Drinks'})

In [ ]:
# Create a new column 'Years of operation' that equals the number of years prior to 2013 that this store was established.
test_df['Years_of_operation'] = 2013 - test_df['Outlet_Establishment_Year']

In [ ]:
# Make category names in the Item_Fat_Content feature consistent.
test_df['Item_Fat_Content'] = test_df['Item_Fat_Content'].replace({'LF' : 'Low Fat', 'reg' : 'Regular', 'low fat' : 'Low Fat'})

# Some "Non-Consumable" item types have a fat content listed. For all "Non-Consumable" items, change the fat content value to 
# a new category called "Not_Edible".
test_df.loc[test_df['Item_Id_Binned'] == 'Non-Consumable', 'Item_Fat_Content'] = 'Not_Edible'

# Ensure the only values in the 'Item_Fat_Content' column are now "Low Fat", "Regular" and "Not_Edible"
# test_df['Item_Fat_Content'].value_counts()

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# Imputing the missing weight values using the average item weight for that item type. 
# -------------------------------------------------------------------------------------------------------------------------

# Create groups of each unique item identifier. For each group, calculate the average item weight.
average_weight_per_item = combined_df.groupby(by="Item_Identifier")['Item_Weight'].mean()

# Create a filter to easily grab all the rows where the Item_Weight value is missing.
missing_weight_filter = (test_df['Item_Weight'].isna())

# Set each missing Item_Weight equal to the average item weight for that Item_Identifier.
# For example, for item 'DRA24' the weight value with be mapped to average_weight_per_item['DRA24'], which is 19.35 per the averages we calculated two lines above.
test_df.loc[missing_weight_filter, 'Item_Weight'] = test_df.loc[missing_weight_filter, 'Item_Identifier'].apply(lambda Item_ID : average_weight_per_item[Item_ID] )

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------
# Imputing the missing Outlet_Size values with the mode (most frequent) outlet size value for that outlet type.
# -------------------------------------------------------------------------------------------------------------------------

# Create groups for each unique value of Outlet_Type. For each outlet_type group, calculate the most frequent outlet_size. 
most_frequent_size_per_type = combined_df.groupby(by="Outlet_Type")['Outlet_Size'].agg(pd.Series.mode)

# Create a filter for easily accessing all the rows where the Outlet_Size value is missing.
missing_outlet_size = test_df['Outlet_Size'].isna()

# impute the missing values in the Outlet_Size column with the most frequent Outlet_Size for that Outlet_Type
test_df.loc[missing_outlet_size, 'Outlet_Size'] = test_df.loc[missing_outlet_size, 'Outlet_Type'].apply(lambda store_type : most_frequent_size_per_type[store_type])

In [ ]:
test_df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Id_Binned               0
Years_of_operation           0
dtype: int64

In [ ]:
filepath = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/Sales_Test_Data_Cleaned_ComboImpute.csv'

test_df.to_csv(path_or_buf=filepath, index=False)

# Dropping the columns that were not used when training the machine learning model.
test_df.drop(columns=['Item_Identifier', 'Outlet_Establishment_Year'], inplace=True)

In [ ]:
test_predictions = gs_results.predict(test_df)

In [ ]:
predictions_df = pd.DataFrame(test_predictions)
predictions_df.rename(columns={0: 'Item_Outlet_Sales'}, inplace=True)

In [ ]:
full_submission_df = pd.concat([submission_columns_df, predictions_df], axis=1)

In [ ]:
filepath = '/content/drive/MyDrive/Programming/Colab Notebooks/Coding_Dojo/Sales_Project/Current/Anderson_Sales_Prediction_Submission2.csv'

full_submission_df.to_csv(path_or_buf=filepath, index=False)